In [1]:
"""Import the required modules"""
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import timeit

Analyzing welding data by welding types

In [2]:
df2 = pd.read_csv('welding_data_by_welding_types.csv')
df2 = pd.get_dummies(df2).drop('welding_type_SAW', axis=1)
df2 = df2[['current',	'voltage',	'welding_speed',	'plate_thickness',
           'welding_type_GMAW',	'welding_type_GTAW',	'welding_type_SMAW',
           'heat_input',	'tensile_strength',	'flexural_strength',	'hardness_value']]
df2.head()

,current,voltage,welding_speed,plate_thickness,welding_type_GMAW,welding_type_GTAW,welding_type_SMAW,heat_input,tensile_strength,flexural_strength,hardness_value
0,160,32,120,6,0,1,0,1536,432.6,236.2,102.5
1,130,32,90,8,0,0,1,2218,332.8,270.2,96.4
2,100,28,120,8,0,1,0,840,382.4,360.8,86.4
3,160,28,90,10,0,1,0,1792,422.8,230.0,105.0
4,100,28,120,8,0,0,0,1400,385.5,276.5,128.4


Exploratory data analysis

In [3]:
# pos = 1
# for i in range(4):
#   print(f'\t{df2.columns[i].capitalize()} vs Response values')
#   plt.figure()
#   for j in range(8, 11):
#     plt.subplot(1, 3, pos)
#     plt.scatter(df2.iloc[:, i], df2.iloc[:, j])
#     plt.ylabel(df2.columns[j])
#     plt.xlabel(df2.columns[i])
#     pos += 1
#   plt.tight_layout()
#   plt.show()
#   print('\n')
#   pos = 1

Generate the necessary variables for modeling

In [4]:
X1 = df2.iloc[:, :7]
X = X1.to_numpy()
y1 = df2.iloc[:, -4:]
y = y1.to_numpy()

In [5]:
def MAD(y, y_pred):
    '''Calculates the Mean Absolute Deviation of two arrays of numbers
    Input: y(array), y_pred(array)
    Output: float
    '''
    import numpy as np

    return np.average(np.abs(y - y_pred))

In [6]:
def MAPE(y, y_pred):
    '''
    Calculates the relationship between the average prediction error
    and the average expected target error for several consecutive periods.

    Input: y(array), y_pred(array)
    Output: float
    '''
    import numpy as np

    # Calculate prediction error
    error = y - y_pred

    return np.average((np.abs(error)/y) * 100)

In [7]:
def TS(y, y_pred):
    '''
    This index is used to track the accuracy of the model

    Input: y(array), y_pred(array)
    Output: float
    '''
    import numpy as np

    def MAD(y, y_pred):
        '''Calculates the Mean Absolute Deviation of two arrays of numbers
        Input: y(array), y_pred(array)
        Output: float
        '''
        return np.average(np.abs(y - y_pred))

    return np.sum(y - y_pred) / MAD(y, y_pred)

Predicting individual response separately to optimize prediction

In [9]:
metrics = []
for h in range(5, 35, 5):
  for i in range(y.shape[1]):
    metric = [y1.columns[i], int(h)]
    cols = y[:, i]
    X_train, X_test, y_train, y_test = train_test_split(X, cols,random_state=1, test_size=0.2)

    tic = timeit.timeit() #Start timer
    sc_X = StandardScaler()
    X_trainscaled=sc_X.fit_transform(X_train)
    X_testscaled=sc_X.transform(X_test)
    reg = MLPRegressor(hidden_layer_sizes=(h,h,h),activation="relu" ,random_state=1, max_iter=1000000).fit(X_trainscaled, y_train)
    y_pred=reg.predict(X_testscaled)
    toc = timeit.timeit() #Stop timer

    # Display data
    print(f'\n{y1.columns[i]}: {int(h)} Nodes')
    a = []
    for j in range(len(y_test)):
        a.append([y_test[j], y_pred[j]])
    a_df = pd.DataFrame(a, columns=['Expected', 'Predicted'])
    display(a_df)

    # Calculate the necessary metrics
    r2 = r2_score(y_pred, y_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mad = MAD(y_test, y_pred)
    mape = MAPE(y_test, y_pred)
    ts = TS(y_test, y_pred)
    time = abs(toc - tic) * 100

    metric.extend([r2, mse, rmse, mad, mape, ts, time])   # Update metric
    metrics.append(metric)  # Undate metrics

metrics_df = pd.DataFrame(metrics, columns=['Variable', 'Nodes', 'R2', 'MSE', 'RMSE', 'MAD', 'MAPE', 'TS', 'Time'])

display(metrics_df)

    # print(f'R^2 for Response variable, {y1.columns[i].capitalize()}: {(r2_score(y_pred, y_test))}')
    # print(f'Expected value: {y_test}')
    # print(f'Predicted value: {y_pred}')


heat_input: 5 Nodes


,Expected,Predicted
0,1120.0,1026.489718
1,1280.0,1407.273805
2,960.0,1207.824686
3,1792.0,1847.501715
4,1024.0,1247.757103
5,2048.0,2177.306745
6,936.0,944.801452
7,768.0,944.204556



tensile_strength: 5 Nodes


,Expected,Predicted
0,312.8,341.097190
1,442.6,341.921266
2,396.5,450.176518
3,422.8,342.852705
4,280.2,350.289602
5,304.2,219.013997
6,394.6,337.176520
7,378.5,337.902514



flexural_strength: 5 Nodes


...


flexural_strength: 30 Nodes


,Expected,Predicted
0,218.4,209.631538
1,238.4,262.690107
2,336.8,377.449379
3,230.0,276.033216
4,196.0,216.959857
5,260.0,206.517235
6,345.0,349.899959
7,394.4,366.694479



hardness_value: 30 Nodes


,Expected,Predicted
0,132.5,117.263596
1,116.2,167.429246
2,90.6,116.296369
3,105.0,98.792757
4,142.2,115.486229
5,110.0,116.259270
6,80.5,107.824850
7,65.0,85.061705


,Variable,Nodes,R2,MSE,RMSE,MAD,MAPE,TS,Time
0,heat_input,5,0.866119,23419.162020,153.033206,132.772543,12.406672,-6.591421,2.324795
1,tensile_strength,5,-0.387129,4665.562049,68.304920,64.487039,17.816970,3.283911,1.391096
2,flexural_strength,5,0.189111,3891.868344,62.384841,55.612855,20.226037,-0.313528,1.320866
3,hardness_value,5,0.012210,574.307909,23.964722,21.976441,21.374712,-1.194076,0.086536
4,heat_input,10,0.978074,4339.585321,65.875529,54.237029,5.478807,2.741821,0.354603
5,tensile_strength,10,0.521435,1746.363739,41.789517,36.951984,10.421916,1.076606,0.438115
6,flexural_strength,10,0.682065,1822.611988,42.692060,34.469013,13.402632,1.724553,0.268351
7,hardness_value,10,-0.249931,1142.616597,33.802612,27.738273,25.983096,-0.000993,0.285460
8,heat_input,15,0.921825,13601.209048,116.624222,102.356847,8.824864,0.392617,0.283333
9,tensile_strength,15,-0.602981,6573.026631,81.074204,67.665888,17.357950,4.456963,0.401534
